In [1]:
import keras
from keras.models import load_model
import random
import numpy as np
import math
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from IPython.display import clear_output
import time
from pandas_datareader import data
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from keras import backend as K
import tensorflow as tf
import ta

Using TensorFlow backend.


In [2]:
# prints formatted price
def formatPrice(n):
	return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

In [3]:
#data = data.DataReader("^DJI", 
                       #start='2000-10-20', 
                       #end='2018-12-31', 
                     #data_source='yahoo')

In [4]:
#data.to_csv("csv/DJI.csv")

In [5]:
X1=pd.read_csv('csv/DJI.csv')[0:1306] #0to1305

In [6]:
#Y1=pd.read_csv('csv/DJI.csv')

In [7]:
#len(Y1)-50

In [8]:
#len(X1)-50

In [9]:
#1256/4527

In [10]:
#X1.tail()

In [11]:
x=X1["Close"][50]

In [12]:
X1['rsi']=ta.momentum.rsi(X1["Close"], n=14, fillna=False)
X1['r']=ta.momentum.wr(X1["High"], X1["Low"], X1["Close"], lbp=14, fillna=False)
X1['cci']=ta.trend.cci(X1["High"], X1["Low"], X1["Close"], n=20, c=0.015, fillna=False) 

In [13]:
def tech(t):
    ls=[0,0,0]
    #0-hold 1-buy 2-sell
    rsi=X1['rsi'][:t+1]
    if(rsi[t]>=70):
        ls[0]=2
    elif (rsi[t]<=30):
         ls[0]=1
    else:
         ls[0]=0
    r=X1['r'][:t+1]
    if(r[t]>=-20):
        ls[1]=2
    elif (r[t]<=-80):
         ls[1]=1
    else:
         ls[1]=0
    cci=X1['cci'][:t+1]
    if(cci[t]<=-100):
        ls[2]=2
    elif (cci[t]>=100):
         ls[2]=1
    else:
         ls[2]=0
    return ls
    

In [14]:
class Portfolio:
    def __init__(self):
        self.cash=x*100
        self.profit=0
        self.inventory = []
        self.nshare=0
        self.last_price=0
        self.sold_share=0
        
    def getData(self):        
        self.vec = []
        self.df=pd.read_csv('csv/DJI.csv')[0:1306]
        return self.df 
    
    def getState(self,data,t):
        self.n=50
        self.res=[]
        for i in range(self.n):                 
            self.res.append(data['Close'][t-i])          
        return np.array([self.res])
        

In [15]:
class RLAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._model()
        self.target_model = self._model()
        self.update_target_model()
      

    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(lr=0.001))
        return model

    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                # a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
                # target[0][action] = reward + self.gamma * t[np.argmax(a)]
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    

In [16]:
#************************-Training-*********************

rl = RLAgent(50,3)
episode_count=100
for e in range(episode_count):
    agent=Portfolio()
    data=agent.getData()
    data['Action']=""
    data['actual_action']=""
    data['share']=""
    data['Cash']=""
    data['trade_profit']=""
    data['Adj_cash']=""
    l = len(data) - 1
    batch_size = 32
    print( "Episode " + str(e+1) + "/" + str(episode_count))
    state = agent.getState(data, 50)
    initial_cash=agent.cash
    total_profit = 0
    total_brokerage=0
    agent.nshare=0
    agent.last_price=0
    data['Action'][50]=0
    data['share'][50]=agent.nshare
    data['Cash'][50]= agent.cash
    profit=0
    pos=""
   
    for t in range(50,l):
        z=tech(t)
        z0=z.count(0)
        z1=z.count(1)
        z2=z.count(2)
        y=[z0,z1,z2]
        z3=y.index(max(y))
        action = rl.act(state)
        data['Action'][t]=0
        data['share'][t]=agent.nshare
        data['Cash'][t]= agent.cash
       
        #print("{} last_p={} : current_p{} :time={} ".format(pos,agent.last_price,data['Close'][t],t))
         
            
        # sit
        next_state = agent.getState(data, t + 1)
        reward = 0
        if action == 0:
            if agent.nshare == 0:
                    data['Adj_cash'][t]=agent.cash
            elif agent.nshare >0:
                    data['Adj_cash'][t]=agent.cash+agent.nshare*data['Close'][t]
            elif agent.nshare <0:
                    data['Adj_cash'][t]=(agent.cash+(-agent.sold_share)*agent.last_price)+(agent.last_price - data['Close'][t])*(-agent.sold_share)
                    
        elif action == 1:# buy
                data['Action'][t]=1
                
                if z3==1:                            
                    if  agent.nshare == 0: #long buy
                        agent.nshare=int(agent.cash/data['Close'][t])
                        agent.cash=agent.cash-agent.nshare*data['Close'][t]
                        agent.last_price=data['Close'][t]
                        data['actual_action'][t]="Bought-open-long"
                        data['share'][t]=agent.nshare
                        data['Cash'][t]= agent.cash
                    elif agent.nshare < 0: #short buy
                        
                        sell_price=agent.last_price
                        ns = agent.nshare
                        agent.nshare=0
                        agent.last_price=0
                        brokerage=((sell_price + data['Close'][t])*(-ns))*0.0010
                        total_brokerage=total_brokerage+brokerage
                        data['trade_profit'][t]=((sell_price - data['Close'][t])*(-ns))-brokerage
                        total_profit = total_profit + ((sell_price - data['Close'][t])*(-ns))-brokerage
                        agent.cash=agent.cash+(-ns*sell_price)+data['trade_profit'][t]
                        data['actual_action'][t]="Bought-closed-short"
                        agent.sold_share=0
                
                    
                    data['Adj_cash'][t]=agent.cash+agent.nshare*data['Close'][t]
                    reward = (1+((data['Close'][t]-data['Close'][t-1])/data['Close'][t-1])*(data['Close'][t-1]/data['Close'][t-50]))
                    data['share'][t]=agent.nshare
                    data['Cash'][t]= agent.cash
                else:
                    if agent.nshare == 0:
                        data['Adj_cash'][t]=agent.cash
                    elif agent.nshare >0:
                        data['Adj_cash'][t]=agent.cash+agent.nshare*data['Close'][t]
                    elif agent.nshare <0:
                        data['Adj_cash'][t]=(agent.cash+(-agent.sold_share)*agent.last_price)+(agent.last_price - data['Close'][t])*(-agent.sold_share)
                                                      
                    
            
        elif action == 2:# sell
            data['Action'][t]=2
            
            if z3==2:  
                    if  agent.nshare == 0: #short sell
                        agent.nshare=int(agent.cash/data['Close'][t])
                             
                        agent.cash=agent.cash-agent.nshare*data['Close'][t]
                        agent.nshare= -agent.nshare
                        agent.sold_share=agent.nshare
                        agent.last_price=data['Close'][t]
                        data['actual_action'][t]="Sold-open-short"
                                
                    elif agent.nshare > 0: 
                        
                        buy_price=agent.last_price
                        ns = agent.nshare
                        agent.nshare=0
                        agent.last_price=0
                        brokerage=((buy_price + data['Close'][t])*(ns))*0.0010
                        data['trade_profit'][t]=(data['Close'][t]-buy_price)*(ns)-brokerage
                        total_profit = total_profit + (data['Close'][t]-buy_price)*(ns)-brokerage
                        agent.cash=agent.cash+(ns)*data['Close'][t]-brokerage
                        data['actual_action'][t]="Sold-closed-long"
               
                    
                    data['Adj_cash'][t]=agent.cash+(-agent.nshare)*agent.last_price+(agent.last_price - data['Close'][t])*(-agent.nshare)
                    reward = -1*(1+((data['Close'][t]-data['Close'][t-1])/data['Close'][t-1])*(data['Close'][t-1]/data['Close'][t-50]))
                    data['share'][t]=agent.nshare
                    data['Cash'][t]= agent.cash
            else:
                if agent.nshare == 0:
                    data['Adj_cash'][t]=agent.cash
                elif agent.nshare >0:
                    data['Adj_cash'][t]=agent.cash+agent.nshare*data['Close'][t]
                elif agent.nshare <0:
                    data['Adj_cash'][t]=(agent.cash+(-agent.nshare)*agent.last_price)+(agent.last_price - data['Close'][t])*(-agent.nshare)
                    
            
                
                    
        done = True if t == l - 1 else False
        rl.remember(state, action, reward, next_state, done)
        state = next_state

        if done:
                rl.update_target_model()
                print( "------------------------------------------")
                print ("Initial Cash: " + formatPrice(initial_cash))
                print ("Final Cash: " + formatPrice(data['Adj_cash'][t]))
                print ("Total  Profit: " + formatPrice(data['Adj_cash'][t]-initial_cash))
                print ("Total % Profit:{} ".format((data['Adj_cash'][t]-initial_cash)/initial_cash*100))
                print("-------------------------------------------")
            
                
            

        if len(rl.memory) > batch_size:
            rl.replay(batch_size)
            
  
    if e % 1 == 0:
        rl.model.save("models/model_ep" + str(e))
        data.to_csv('csv/after_training.csv')

Episode 1/100


/home/jagdish/anaconda3/envs/RL/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jagdish/anaconda3/envs/RL/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jagdish/anaconda3/envs/RL/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jagdish/anaconda3/envs/RL/lib/python3.7/site-p

/home/jagdish/anaconda3/envs/RL/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jagdish/anaconda3/envs/RL/lib/python3.7/site-packages/ipykernel_launcher.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jagdish/anaconda3/envs/RL/lib/python3.7/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jagdish/anaconda3/envs/RL/lib/python3.7/site-

------------------------------------------
Initial Cash: $1064615.04
Final Cash: $1785538.49
Total  Profit: $720923.45
Total % Profit:67.71681980594147 
-------------------------------------------
Episode 2/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $1001362.50
Total  Profit: -$63252.54
Total % Profit:-5.941352784961697 
-------------------------------------------
Episode 3/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $1375267.25
Total  Profit: $310652.21
Total % Profit:29.179769546063554 
-------------------------------------------
Episode 4/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $1327869.47
Total  Profit: $263254.43
Total % Profit:24.727663692073264 
-------------------------------------------
Episode 5/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $1129887.82
Total  Profit: $65272.78
Total % Profit:6.131115710105714 
-

Episode 40/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $1058210.04
Total  Profit: -$6405.00
Total % Profit:-0.6016255923640368 
-------------------------------------------
Episode 41/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $1100681.90
Total  Profit: $36066.86
Total % Profit:3.387784201082088 
-------------------------------------------
Episode 42/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $1375777.52
Total  Profit: $311162.48
Total % Profit:29.227698961203874 
-------------------------------------------
Episode 43/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $979418.60
Total  Profit: -$85196.44
Total % Profit:-8.002557980630092 
-------------------------------------------
Episode 44/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $1542225.63
Total  Profit: $477610.59
Total % Profit:44

Episode 79/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $1001247.58
Total  Profit: -$63367.45
Total % Profit:-5.952147262920922 
-------------------------------------------
Episode 80/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $1272429.70
Total  Profit: $207814.66
Total % Profit:19.520169728494498 
-------------------------------------------
Episode 81/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $1194301.82
Total  Profit: $129686.78
Total % Profit:12.181565409591826 
-------------------------------------------
Episode 82/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $1257176.71
Total  Profit: $192561.67
Total % Profit:18.08744616910564 
-------------------------------------------
Episode 83/100
------------------------------------------
Initial Cash: $1064615.04
Final Cash: $852202.56
Total  Profit: -$212412.48
Total % Profit: